In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import os
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import models
from keras.callbacks import EarlyStopping, CSVLogger, LearningRateScheduler, ModelCheckpoint
from keras.layers import Input, Dense, Dropout, Flatten, LSTM, GRU, BatchNormalization, GlobalAveragePooling1D
from keras.models import load_model, Model
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [3]:
path='/content/drive/MyDrive/Capstone_Design/EEG_DATA_Shuffle_Train_500.csv'
dataDF = pd.read_csv(path)
dataDF = dataDF.drop('Unnamed: 0', axis=1)
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,-613.9531,8.3703,-176.9841,-219.4107,-0.5328,94.9314,-39.4071,130.4078,-113.2378,-4.4257,...,-84.1440,126.1760,-23.1316,-34.8843,-114.5272,-617.7455,-127.1501,-235.2947,-181.4069,0.0
1,-31.4229,-116.7955,-106.9859,-28.1580,-28.7239,-69.4110,-70.9967,-41.4681,-3.3877,-72.3992,...,-3.2703,-9.4238,-17.9684,-0.1850,-5.6882,-37.3602,-34.1668,-1.4736,38.9844,3.0
2,19.2428,39.7509,45.2220,22.9375,43.3171,52.6086,37.2702,7.9396,45.4657,10.6151,...,11.0563,42.4707,14.7642,4.5331,26.1308,36.0465,47.0181,6.1247,-21.7909,0.0
3,23.5423,55.7871,-1.9299,-58.4906,-1.2527,2.6059,4.0022,-2.4664,55.1775,-3.0306,...,-16.8038,-31.0302,-38.5513,15.6749,-21.4778,-36.8297,-23.9080,-49.7100,-139.1322,3.0
4,-9.8380,-14.6411,-17.4022,-23.3906,-5.4970,-5.4633,-5.6305,-27.2574,-8.3492,-10.9784,...,-18.1588,-14.0856,-11.9353,-13.7297,-9.6201,-21.1847,-14.3220,-10.4927,16.7754,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,14.3461,-1.6934,-21.1162,-12.7199,-10.6491,-5.0185,-13.5567,-7.7491,-5.5391,24.0267,...,28.5637,-24.4292,-26.2400,-12.7488,-1.2411,43.8751,14.2213,-28.3929,-16.0004,1.0
2156,-5.8924,21.1051,3.8857,0.6180,5.5196,31.6711,17.1194,12.5737,37.4985,4.4398,...,2.8834,-14.6937,-13.9947,-1.7489,-11.3992,8.0760,-10.1007,-11.8973,-22.9405,1.0
2157,26.6818,105.7402,107.7274,94.4367,7.3311,34.8395,11.8341,1.2366,-32.5458,14.6907,...,18.8723,10.7117,25.4779,37.4412,-0.1916,9.7069,10.9658,-2.7385,4.1386,3.0
2158,11.5242,2.1610,32.1919,-3.1740,-0.8422,-6.3219,-2.0782,-7.7142,0.9459,-20.0408,...,-8.2425,-3.9377,0.2258,-2.9644,0.6710,-3.2271,-10.7153,-3.1265,2.1440,1.0


In [4]:
ct = 500

x_train = []
y_train = []

print(len(dataDF))

for i in range(len(dataDF)):
    print(i)
    x_train.append([])
    x = 0
    y = 20
    one_row_data = dataDF.iloc[i]
    y_train.append(one_row_data[y - 1])
    for j in range(ct):
        x_train[i].append([one_row_data[x:y - 1]])
        x += 20
        y += 20

x_train = np.array(x_train).reshape(len(x_train), ct, 19)
y_train = np.array(pd.get_dummies(y_train))

2160
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [5]:
print("Training set shapes:")
print(x_train.shape)
print(y_train.shape)

Training set shapes:
(2160, 500, 19)
(2160, 4)


In [6]:
path='/content/drive/MyDrive/Capstone_Design/EEG_DATA_Shuffle_Validation_500.csv'
dataDF = pd.read_csv(path)
dataDF = dataDF.drop('Unnamed: 0', axis=1)
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,17.8702,21.4625,7.5565,-27.3715,2.4686,4.5210,-4.7260,-12.1676,2.2848,-16.4927,...,16.4425,27.0572,-11.4758,4.2899,27.0235,-7.7163,-24.5729,3.5688,-1.5061,1.0
1,-34.6827,-13.2270,28.4124,37.2452,-28.3499,-30.9165,18.4352,-21.4872,18.3771,27.5069,...,-30.0969,13.4330,27.0925,-14.5877,46.8918,3.2198,19.4962,38.6907,-53.4658,0.0
2,45.8402,-10.8951,12.5457,-11.4132,-20.6368,-7.4448,1.8782,2.4676,-0.4508,-3.3592,...,5.8944,14.8315,14.7317,-22.4161,19.7682,6.8033,9.7496,9.5028,4.6837,3.0
3,-76.7901,10.4661,60.9218,39.1930,33.3773,65.0877,49.0782,5.1342,43.5867,0.9796,...,-38.8314,-50.8898,-42.2924,-22.9204,-48.7766,-44.7011,-38.4557,-40.1042,-36.8672,1.0
4,4.3084,92.5663,84.6904,46.8202,48.0335,51.5039,53.3825,32.5205,93.0373,2.1225,...,14.2065,10.0970,24.9994,33.3981,-4.9856,43.4911,34.7056,-2.6518,-3.7837,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,-3.4889,-7.5468,5.6223,-42.8946,-0.5838,-7.7837,-24.9255,-8.1583,-34.5417,-12.6059,...,45.4875,49.4695,27.1290,62.0512,47.4708,13.4493,24.4128,29.2915,37.7464,0.0
320,-7.9270,-6.4717,41.8189,2.2906,1.1288,13.5942,-28.7225,-5.6220,1357.4370,-0.0739,...,-15.9046,-15.5793,-21.8264,-35.8572,-5.2574,-8.3024,-20.7946,-4.7600,-6.7358,2.0
321,-4708.2950,10.6987,20.6212,-31.9737,0.1758,17.5586,28.2019,14.6711,64.9319,3.1740,...,-0.5512,-46.3319,-28.5157,7.1775,-11.4533,-31.7425,-45.8124,-20.0943,-45.0598,0.0
322,18.4628,10.3464,-8.5795,-0.4097,-29.0324,-27.6057,36.3050,-17.4636,-38.1526,12.4380,...,6.8641,2.3899,0.9938,75.9316,-2.2810,8.5736,37.7211,-8.4882,16.3725,2.0


In [7]:
ct = 500

x_val = []
y_val = []

print(len(dataDF))

for i in range(len(dataDF)):
    print(i)
    x_val.append([])
    x = 0
    y = 20
    one_row_data = dataDF.iloc[i]
    y_val.append(one_row_data[y - 1])
    for j in range(ct):
        x_val[i].append([one_row_data[x:y - 1]])
        x += 20
        y += 20

x_val = np.array(x_val).reshape(len(x_val), ct, 19)
y_val = np.array(pd.get_dummies(y_val))

324
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [8]:
print("Validation set shapes:")
print(x_val.shape)
print(y_val.shape)

Validation set shapes:
(324, 500, 19)
(324, 4)


In [9]:
path='/content/drive/MyDrive/Capstone_Design/EEG_DATA_Shuffle_Test_500.csv'
dataDF = pd.read_csv(path)
dataDF = dataDF.drop('Unnamed: 0', axis=1)
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,10.4428,-6.2617,-17.0080,-26.7763,-2.7437,-8.7904,-1.2462,4.5911,0.7072,5.4368,...,25.0777,-0.0593,21.1675,4.0514,9.1570,17.9326,24.1978,32.4984,34.7287,1.0
1,27.5277,9.1181,-5.3722,15.0280,-11.4663,5.6988,-3.4859,49.7776,25.7247,-20.5147,...,23.8738,43.8601,13.6545,0.7813,31.9035,52.5582,37.7786,-17.5178,13.9098,1.0
2,3.5141,-46.9753,-42.6202,-10.6172,-6.7741,-12.3721,-12.9584,-55.8731,-11.5008,11.3118,...,-11.1626,-5.3945,-22.7586,16.5264,-6.6955,5.2774,5.3839,-28.1251,-27.5721,2.0
3,13.1853,9.5953,59.0312,14.6561,14.5705,-1.0250,-0.8357,9.1998,-5.4283,-10.0039,...,-0.0823,2.1325,-0.8022,26.1801,8.8764,0.2503,6.8797,3.6670,4.3527,3.0
4,14.2301,31.4379,32.5355,9.3517,20.1011,28.8178,25.1274,7.3163,1.7966,6.4908,...,-6.5038,-7.1040,-14.3001,10.0207,-5.7362,-6.8697,-6.8623,-4.3336,-4.5961,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,2.9216,-20.6072,-8.2800,-13.4839,-2.1357,0.4423,-0.6483,-3.5984,7.6571,-18.1392,...,26.0548,22.5790,17.8109,15.7244,18.7196,34.5459,21.9128,18.4194,11.4716,1.0
212,-19.6713,-28.8127,-14.9236,2.2462,-2.1703,-3.0315,19.7007,0.3452,-12.7320,2.0537,...,-12.2667,13.6936,-0.2940,-1.6649,-5.2100,-2.0658,-10.6158,-12.2835,-13.5389,0.0
213,25.8002,7.1440,21.1307,17.3725,18.2719,13.6507,11.7411,22.2569,23.6596,21.1655,...,32.2828,-4.7560,-5.1947,-0.0218,7.3851,13.6946,-5.2101,7.4650,15.2721,1.0
214,-19.2564,3.2113,9.0956,-16.0042,3.6467,-14.6631,-8.1552,-8.3088,-12.6606,-17.5448,...,2.4161,-0.9172,16.8690,16.0995,15.8436,5.3916,-5.5050,27.4403,19.4462,3.0


In [10]:
ct = 500

x_test = []
y_test = []

print(len(dataDF))

for i in range(len(dataDF)):
    print(i)
    x_test.append([])
    x = 0
    y = 20
    one_row_data = dataDF.iloc[i]
    y_test.append(one_row_data[y - 1])
    for j in range(ct):
        x_test[i].append([one_row_data[x:y - 1]])
        x += 20
        y += 20

x_test = np.array(x_test).reshape(len(x_test), ct, 19)
y_test = np.array(pd.get_dummies(y_test))

216
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215


In [11]:
print("Test set shapes:")
print(x_test.shape)
print(y_test.shape)

Test set shapes:
(216, 500, 19)
(216, 4)


In [24]:
# i_lstm = Input(shape=(x_train.shape[1], x_train.shape[2]))

# x_lstm = LSTM(128, return_sequences=True)(i_lstm)
# x_lstm = Dropout(0.2)(x_lstm)

# x_pool = GlobalAveragePooling1D()(x_lstm)

# x_dense = Dense(64, activation='relu')(x_pool)
# x_dense = Dropout(0.2)(x_dense)

# y_output = Dense(4, activation='softmax')(x_dense)
# myMdl = Model(i_lstm, y_output)

# myMdl.summary()

In [31]:
i_lstm = Input(shape=(x_train.shape[1], x_train.shape[2]))

x_lstm = LSTM(256, return_sequences=True)(i_lstm)
x_lstm = BatchNormalization()(x_lstm)
x_lstm = Dropout(0.3)(x_lstm)

x_lstm = LSTM(128, return_sequences=True)(x_lstm)
x_lstm = BatchNormalization()(x_lstm)
x_lstm = Dropout(0.3)(x_lstm)

x_lstm = LSTM(64, return_sequences=True)(x_lstm)
x_lstm = BatchNormalization()(x_lstm)
x_lstm = Dropout(0.3)(x_lstm)

x_pool = GlobalAveragePooling1D()(x_lstm)

x_dense = Dense(128, activation='relu')(x_pool)
x_dense = BatchNormalization()(x_dense)
x_dense = Dropout(0.3)(x_dense)

x_dense = Dense(64, activation='relu')(x_dense)
x_dense = BatchNormalization()(x_dense)
x_dense = Dropout(0.2)(x_dense)

y_output = Dense(4, activation='softmax')(x_dense)
myMdl = Model(i_lstm, y_output)

myMdl.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 500, 19)]         0         
                                                                 
 lstm_10 (LSTM)              (None, 500, 256)          282624    
                                                                 
 batch_normalization (Batch  (None, 500, 256)          1024      
 Normalization)                                                  
                                                                 
 dropout_17 (Dropout)        (None, 500, 256)          0         
                                                                 
 lstm_11 (LSTM)              (None, 500, 128)          197120    
                                                                 
 batch_normalization_1 (Bat  (None, 500, 128)          512       
 chNormalization)                                          

In [30]:
# i_lstm = Input(shape=(x_train.shape[1], x_train.shape[2]))

# x_lstm = LSTM(256, return_sequences=True)(i_lstm)
# x_lstm = Dropout(0.3)(x_lstm)

# x_lstm = LSTM(128, return_sequences=True)(x_lstm)
# x_lstm = Dropout(0.3)(x_lstm)
# x_lstm = LSTM(64, return_sequences=True)(x_lstm)
# x_lstm = Dropout(0.3)(x_lstm)

# x_pool = GlobalAveragePooling1D()(x_lstm)

# x_dense = Dense(128, activation='relu')(x_pool)
# x_dense = Dropout(0.3)(x_dense)

# x_dense = Dense(64, activation='relu')(x_dense)
# x_dense = Dropout(0.2)(x_dense)

# y_output = Dense(4, activation='softmax')(x_dense)
# myMdl = Model(i_lstm, y_output)

# myMdl.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 500, 19)]         0         
                                                                 
 lstm_7 (LSTM)               (None, 500, 256)          282624    
                                                                 
 dropout_12 (Dropout)        (None, 500, 256)          0         
                                                                 
 lstm_8 (LSTM)               (None, 500, 128)          197120    
                                                                 
 dropout_13 (Dropout)        (None, 500, 128)          0         
                                                                 
 lstm_9 (LSTM)               (None, 500, 64)           49408     
                                                                 
 dropout_14 (Dropout)        (None, 500, 64)           0   

In [32]:
root = '/content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/'
folder_name = '0417_1modelsave'
os.mkdir(root+folder_name)
csv_logger = CSVLogger(root+folder_name+'/training.csv', separator=",", append=True)
mcp_save_best = tf.keras.callbacks.ModelCheckpoint(
    root+folder_name+'/{epoch:02d}-{val_loss:.5f}_best', save_best_only=True,
    monitor='val_loss', verbose=1, mode='min')
mcp_save = tf.keras.callbacks.ModelCheckpoint(
    root+folder_name+'/{epoch:02d}-{val_loss:.5f}', save_best_only=False,
    monitor='val_loss', verbose=1, mode='min')

In [33]:
adam = Adam(learning_rate=0.001)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
mc_loss = ModelCheckpoint('/content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/best_lstm_loss_v1.h5', monitor='val_loss', verbose=1, save_best_only=True)
mc = ModelCheckpoint('/content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/best_lstm_acc_v1.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
lr_schedule = LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))

myMdl.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])

lstm_h = myMdl.fit(x_train, y_train,
                   batch_size=32,
                   validation_data=(x_val, y_val),
                   epochs=200,
                   callbacks=[es, mc, mc_loss, mcp_save_best, mcp_save, lr_schedule, csv_logger])

Epoch 1/200
68/68 [==============================] - ETA: 0s - loss: 0.8876 - accuracy: 0.6463
Epoch 1: val_accuracy improved from -inf to 0.46605, saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/best_lstm_acc_v1.h5

Epoch 1: val_loss improved from inf to 1.24237, saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/best_lstm_loss_v1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 1: val_loss improved from inf to 1.24237, saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/0417_1modelsave/01-1.24237_best

Epoch 1: saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/0417_1modelsave/01-1.24237
68/68 [==============================] - 40s 478ms/step - loss: 0.8876 - accuracy: 0.6463 - val_loss: 1.2424 - val_accuracy: 0.4660 - lr: 0.0010
Epoch 2/200
68/68 [==============================] - ETA: 0s - loss: 0.2587 - accuracy: 0.9097
Epoch 2: val_accuracy improved from 0.46605 to 0.51852, saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/best_lstm_acc_v1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.24237 to 1.20721, saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/best_lstm_loss_v1.h5

Epoch 2: val_loss improved from 1.24237 to 1.20721, saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/0417_1modelsave/02-1.20721_best

Epoch 2: saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/0417_1modelsave/02-1.20721
68/68 [==============================] - 29s 427ms/step - loss: 0.2587 - accuracy: 0.9097 - val_loss: 1.2072 - val_accuracy: 0.5185 - lr: 9.0484e-04
Epoch 3/200
68/68 [==============================] - ETA: 0s - loss: 0.1499 - accuracy: 0.9463
Epoch 3: val_accuracy did not improve from 0.51852

Epoch 3: val_loss did not improve from 1.20721

Epoch 3: val_loss did not improve from 1.20721

Epoch 3: saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/0417_1modelsave/03-1.30370
68/68 [==============================] - 16s 233ms/step - loss: 0.1499 - accuracy: 0.9463 - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 4: val_loss did not improve from 1.20721

Epoch 4: val_loss did not improve from 1.20721

Epoch 4: saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/0417_1modelsave/04-1.54871
68/68 [==============================] - 20s 295ms/step - loss: 0.0849 - accuracy: 0.9634 - val_loss: 1.5487 - val_accuracy: 0.5772 - lr: 7.4082e-04
Epoch 5/200
68/68 [==============================] - ETA: 0s - loss: 0.0626 - accuracy: 0.9741
Epoch 5: val_accuracy did not improve from 0.57716

Epoch 5: val_loss did not improve from 1.20721

Epoch 5: val_loss did not improve from 1.20721

Epoch 5: saving model to /content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/0417_1modelsave/05-1.88896
68/68 [==============================] - 16s 238ms/step - loss: 0.0626 - accuracy: 0.9741 - val_loss: 1.8890 - val_accuracy: 0.5093 - lr: 6.7032e-04
Epoch 6/200
68/68 [==============================] - ETA: 0s - loss: 0.0486 - accuracy: 0.9773
Epoch 6: val_accuracy did not improve from 0.57716

E

In [35]:
lstm_best = load_model('/content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/best_lstm_acc_v1.h5')
lstm_accuracy = lstm_best.evaluate(x_test, y_test, verbose=1)[1]
print("Test Acc. lstm: {:.3f}%".format(lstm_accuracy * 100))

lstm_best_loss = load_model('/content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/best_lstm_loss_v1.h5')
lstm_loss = lstm_best_loss.evaluate(x_test, y_test, verbose=1)[0]
print("Test val_Loss. lstm:{:.3f}".format(lstm_loss))

y_pred = np.array(list(map(lambda x: np.argmax(x), lstm_best.predict(x_test))))
y_test = y_test.argmax(axis=1)
cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred, digits=4))

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1919, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr

In [ ]:
y_pred_one_hot = lstm_best.predict(x_test)
y_pred = np.argmax(y_pred_one_hot, axis=1)

lb = LabelBinarizer()
lb.fit(y_test)
y_test_one_hot = lb.transform(y_test)

fpr = dict()
tpr = dict()
roc_auc = dict()
class_labels = ['Eyes Open', 'Eyes Closed', 'Waving hand', 'Mental arithmetic']

for i in range(len(class_labels)):
    fpr[i], tpr[i], _ = roc_curve(y_test_one_hot[:, i], y_pred_one_hot[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(class_labels))]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(len(class_labels)):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= len(class_labels)

fpr_macro = all_fpr
tpr_macro = mean_tpr
roc_auc_macro = auc(fpr_macro, tpr_macro)

plt.figure()
plt.plot(fpr_macro, tpr_macro,
         label='macro-average (area = {0:0.4f})'
               ''.format(roc_auc_macro),
         color='navy', linestyle=':', linewidth=4)

for i in range(len(class_labels)):
    plt.plot(fpr[i], tpr[i], lw=2,
             label='{0} (area = {1:0.4f})'
             ''.format(class_labels[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LSTM - ROC CURVE')
plt.legend(loc="lower right")
plt.show()

In [ ]:
myMdl = tf.keras.models.load_model('/content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/20240415_3modelsave/24-0.16987_best')

In [ ]:
history = pd.read_csv('/content/drive/MyDrive/Capstone_Design/LSTM_Save_Model/20240415_3modelsave/training.csv')

In [ ]:
acc = lstm_h.history['accuracy']
val_acc = lstm_h.history['val_accuracy']
loss = lstm_h.history['loss']
val_loss = lstm_h.history['val_loss']
plt.figure(figsize=(12, 12))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color='r')
plt.plot(val_acc, label='Validation Accuracy', color='b', linestyle='--')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right', fontsize=13)
plt.ylabel('Accuracy', fontsize=16, weight='bold')
plt.title('LSTM - Training & Validation Acc.', fontsize=16, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='r')
plt.plot(val_loss, label='Validation Loss', color='b', linestyle='--')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=13)
plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
plt.title('LSTM - Training & Validation Loss', fontsize=15, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')
plt.show()
